In [1]:
import requests
from bs4 import BeautifulSoup as bs4
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import getpass
import easygui as eg
from tqdm import tqdm

In [2]:
def click(x):
    button = driver.find_element(By.XPATH, x)
    button.click()
    time.sleep(2)

### scraping from stepstone

scraping from stepstone.de to get page source of the job results pages (page 1, page 2, page 3, etc.)

In [3]:
url = "https://www.stepstone.de/jobs/python-data-analyst"

options = Options()
# options.add_argument("--disable-notifications")
# to prevent from being spotted as a robot
options.add_argument('--disable-gpu')
options.add_argument('user-agent=fake-useragent')
# installing chromedriver, so that we dont need to keep the chromedriver file
# that needs to be updated every once in a while. better install the latest automatically
driver = webdriver.Chrome(ChromeDriverManager().install())
# opens the browser, maximize window size
# but its often problematic if run in mac. so you can comment this one out
# driver.maximize_window()
# opening url
driver.get(url)
time.sleep(6)
ok1 = "/html/body/div[10]/section/div/section/div[2]/div[1]/div[2]/div"
ok2 = "/html/body/div[9]/section/div/section/div[2]/div[1]/div[2]/div"
ok3 = "/html/body/div[10]/section/div/section/div[2]/div[1]/div[2]"
try:
    click(ok1)
except:
    pass
try:
    click(ok2)
except:
    pass
try:
    click(ok3)
except:
    pass
npage = "/html/body/div[4]/div[1]/div/div/div[2]/div/div[2]/div[3]/div/nav/ul/li[9]/a"
html1 = driver.page_source
search_result_1 = []
end = False
while not end:
    search_result_1.append(html1)
    time.sleep(1)
    i = 0
    while i < 4:
        driver.execute_script("window.scrollTo(0, window.scrollY + 1500)") 
        time.sleep(.2)
        i+=1
    time.sleep(4)
    click(npage)
    html2 = driver.page_source
    # comparing html_before and html_after
    if html1 == html2:
        end = True
    else:
        html1 = html2

[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:01<00:00, 6.74MB/s]
C:\Users\mdima\AppData\Local\Temp\ipykernel_32596\2352680371.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [5]:
job_url_1 = []
for h in tqdm(search_result_1):
    soup = bs4(h)
    res = soup.find_all("article",{"class":"resultlist-1jx3vjx"})
    for r in res:
        href = r.find("a",{"class":"resultlist-1uvdp0v"}).get("href")
        href = "https://www.stepstone.de"+href
        job_url_1.append(href)
print(len(job_url_1))
df_1 = pd.DataFrame(columns = ["job_url"])
df_1["job_url"] = job_url_1
print(df_1.shape)
df_1.head()

100%|██████████| 337/337 [00:41<00:00,  8.03it/s]

8405
(8405, 1)


,job_url
0,https://www.stepstone.de/stellenangebote--Prod...
1,https://www.stepstone.de/stellenangebote--Data...
2,https://www.stepstone.de/stellenangebote--Data...
3,https://www.stepstone.de/stellenangebote--Data...
4,https://www.stepstone.de/stellenangebote--Data...


In [6]:
df_1.to_csv("job url 1.csv", index = False)

In [ ]:
df_1 = pd.read_csv("job url 1.csv")

In [9]:
# splitting the csv file above into smaller csv files
def split_file(df):
    fileindex = 1
    for i in range(0,8501,500):
        a = df.iloc[i:i+500] # each smaller file contains 500 rows
        filename = str(fileindex)+".csv"
        a.to_csv(filename, index = False)
        fileindex += 1

In [10]:
# i needed to split the file so other students could help me with scraping. the time was too short to scrape everything by myself
split_file(df_1)

In [17]:
# this function reads different csv files containing page urls. 
# then get only the page source. then create csv files for each corresponding url files
# it will be concatenated later on
def get_page_source(a,b):
    for f in tqdm(range(a,b)): # 17 is the number of split files
        source = []
        filename = str(f)+".csv" # defining file name
        df_url = pd.read_csv("./"+filename)
        job_url = df_url["job_url"].tolist()
        options = Options()
        # installing chromedriver, so that we dont need to keep the chromedriver file
        # that needs to be updated every once in a while. better install the latest automatically
        driver = webdriver.Chrome(ChromeDriverManager().install())
        for i in job_url: # get full page source for each job offer page
            # options.add_argument("--disable-notifications")
            # to prevent from being spotted as a robot
            options.add_argument('--disable-gpu')
            options.add_argument('user-agent=fake-useragent')
            # opens the browser, maximize window size
            # opening url
            driver.get(i)
            time.sleep(3.5)
            page = driver.page_source
            source.append(page)
        # saving source column separately for each splitted file
        job_source = pd.DataFrame(columns = ["job_source"])
        job_source["job_source"] = source
        filename = str(f)+" source.csv"
        job_source.to_csv("./"+filename, index = False)

DONT RUN THIS: TOO HEAVY AND TAKES TOO MUCH TIME

In [21]:
# put a pair of number from 0 to 18 as parameters. 
# if you want to do all in one go, then put 0,18
# if you want to split the job, put 0,3 then 3,5 then 5,8 and so on
# the max number was set to 18. you can change it in the previous cell
# successfully scraped until: 4
get_page_source(5,7)

getting some details from each page source we had from the big scraping job (see cell above)

In [47]:
df_stepstone = pd.DataFrame()
url_1,d_1,t_1,cn_1,cl_1,c_1= [],[],[],[],[],[]
files = [1,3,4,5,7,8,9,10,12,13,14]
for f in tqdm(files):
    filename1 = str(f)+".csv"
    filename2 = str(f)+" source.csv"
    df1 = pd.read_csv("./"+filename1)
    df2 = pd.read_csv("./"+filename2)
    description_1,title_1,comp_name_1,comp_url_1,city_1 = [],[],[],[],[]
    source = df2["source"].tolist()
    for s in source:
        soup = bs4(s, "html.parser")
        try:
            jobtitle = soup.find("span",{"data-at":"header-job-title"}).text
        except:
            jobtitle = "unknown"
        title_1.append(jobtitle)
        try:
            compname = soup.find("a",{"data-at":"header-company-name"}).text
        except:
            compname = "unknown"
        comp_name_1.append(compname)
        try:
            complink = soup.find("a",{"data-at":"header-company-name"}).get("href")
        except:
            complink = "unknown"
        comp_url_1.append(complink)
        try:
            city = soup.find("span",{"class":"listing-content-provider-1u79rpn"}).text
        except:
            city = "unknown"
        city_1.append(city)
        infotext = soup.find_all("div",{"class":"listing-content-provider-10ltcrf"})
        desc = []
        for i in infotext:
            try:
                texts = i.find_all("p")
                for t in texts:
                    info = t.text
                    desc.append(info)
            except:
                pass
            try:
                texts = i.find_all("li")
                for t in texts:
                    info = t.text
                    desc.append(info)
            except:
                pass
        # enemy spotted
        description = " ".join(desc).replace("\xa0","").replace("\\n","")
        description_1.append(description)
    d_1.extend(description_1)
    t_1.extend(title_1)
    cn_1.extend(comp_name_1)
    cl_1.extend(comp_url_1)
    c_1.extend(city_1)
    url_1.extend(df1["href_to_scrape"].tolist())

100%|██████████| 11/11 [34:16<00:00, 186.93s/it]


In [36]:
x = 0
for i in t_1:
    if i != None:
        x+=1
x

5500

In [48]:
df_stepstone["job_url"] = url_1
df_stepstone["description"] = d_1
df_stepstone["job_title"] = t_1
df_stepstone["comp_name"] = cn_1
df_stepstone["comp_link"] = cl_1
df_stepstone["city"] = c_1
df_stepstone.shape

(5500, 6)

In [41]:
import numpy as np

In [49]:
for i in df_stepstone.columns:
    df_stepstone[i] = df_stepstone[i].replace("unknown", np.nan)
print(df_stepstone.shape)
df_stepstone = df_stepstone.dropna(subset = "city")
print(df_stepstone.shape)


(5500, 6)
(4780, 6)


In [50]:
df_stepstone.to_csv("stepstone 1 incomplete.csv", index = False)

### now scrape from indeed.com

In [45]:
from selenium.webdriver.common.keys import Keys

In [ ]:
url2 = "https://de.indeed.com/jobs?q=python+data+analyst&l=&from=searchOnHP&vjk=79940d31c430a131"
options = Options()
# options.add_argument("--disable-notifications")
# to prevent from being spotted as a robot
options.add_argument('--disable-gpu')
options.add_argument('user-agent=fake-useragent')
# installing chromedriver, so that we dont need to keep the chromedriver file
# that needs to be updated every once in a while. better install the latest automatically
driver = webdriver.Chrome(ChromeDriverManager().install())
# opens the browser, maximize window size
driver.maximize_window()
# opening url
driver.get(url2)
time.sleep(2)
html1 = driver.page_source
npage = "/html/body/main/div/div[1]/div/div/div[5]/div[1]/nav/div[6]/a"
lhtml2 = []
end = False
counter = 1
while not end:
    if counter == 2:
        time.sleep(2)
        webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
    lhtml2.append(html1)
    time.sleep(1)
    i = 0
    while i < 4:
        driver.execute_script("window.scrollTo(0, window.scrollY + 1500)") 
        time.sleep(.2)
        i+=1
    time.sleep(1)
    try:
        click(npage)
    except:
        end = True
    counter += 1
    html2 = driver.page_source
    # comparing html_before and html_after
    if html1 == html2:
        end = True
    else:
        html1 = html2

In [ ]:
lhref2 = []
for h in tqdm(lhtml2):
    soup = bs4(h)
    container = soup.find("ul",{"class":"jobsearch-ResultsList css-0"})
    res = container.find_all("div",{"class":"slider_container css-g7s71f eu4oa1w0"})
    for r in res:
        href = r.find("h2",{"tabindex":"-1"}).find("a").get("href")
        href = "https://de.indeed.com" + href
        lhref2.append(href)
print(len(lhref2))
href_pd2 = pd.DataFrame(columns = ["job_url"])
href_pd2["job_url"] = lhref2
# href_pd2.to_csv("job url 2.csv", index = False)
print(href_pd2.shape)
href_pd2.head()

In [ ]:
href_pd2 = pd.read_csv("job url 2.csv")

In [ ]:
source2 = []
options = Options()
# installing chromedriver, so that we dont need to keep the chromedriver file
# that needs to be updated every once in a while. better install the latest automatically
driver = webdriver.Chrome(ChromeDriverManager().install())
for i in tqdm(lhref2):
    # options.add_argument("--disable-notifications")
    # to prevent from being spotted as a robot
    options.add_argument('--disable-gpu')
    options.add_argument('user-agent=fake-useragent')
    # opens the browser, maximize window size
    driver.maximize_window()
    # opening url
    driver.get(i)
    time.sleep(1)
    page = driver.page_source
    source2.append(page)
href_pd2["job_source"] = source2
# href_pd2.to_csv("job url 2.csv", index = False)

In [ ]:
href_pd2 = pd.read_csv("job url 2.csv")
href_pd2.head(1)

In [ ]:
ldescription2,ltitle2,lcompname2,lcomplink2,lcity2 = [],[],[],[],[]

for s in tqdm(source2):
    soup = bs4(s, "html.parser")
    try:
        jobtitle = soup.find("h1",{"class":"icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title"}).text
    except:
        jobtitle = None
    ltitle2.append(jobtitle)
    try:
        compname = soup.find("div",{"data-company-name":"true"}).text
    except:
        compname = None
    lcompname2.append(compname)
    try:
        complink = soup.find("div",{"data-company-name":"true"}).find("a").get("href")
    except:
        complink = None
    lcomplink2.append(complink)
    city = None
    lcity2.append(city)
    infotext = soup.find("div",{"id":"jobDescriptionText"}).find_all("p")
    desc = []
    for i in infotext:
        try:
            texts = i.find("b").text
            desc.append(texts)
        except:
            texts = i.text
        desc.append(texts)
    # enemy spotted
    description = " ".join(desc)
    ldescription2.append(description)
print(len(ldescription2), len(source2))

In [ ]:
href_pd2["description"] = ldescription2
href_pd2["job_title"] = ltitle2
href_pd2["comp_name"] = lcompname2
href_pd2["comp_link"] = lcomplink2
href_pd2["city"] = lcity2
# href_pd2.to_csv("job url 2.csv", index = False)

In [ ]:
href_pd2 = pd.read_csv("job url 2.csv") # sorry about the filename. i will change it to a better one if i got time
href_pd2.head(1)

In [ ]:
full = pd.concat([href_pd, href_pd2], axis = 0)
# full.to_csv("full.csv", index = False)